# Expected Performance Tests

In this notebook, I will be focusing on understanding and using a ResNet.

In order to ascertain the expected performance of the (later developed) CNN, I will be using the aforementioned ResNet for classification on the raw dataset. This will allow for a fair comparison/tradeoff between understanding if my developed model is under/over performing.

In other words, the transfer learning results will be used as a performance reference.

In [40]:
import pandas as pd
import numpy as np
import random

## Why ResNet?

### Concept

The concept of a Residual Network was conceived when dealing with the vanishing gradient problem related to Deep Neural Networks, an issue that led to inneficient/irrelevant weight updates, significantly increasing training time and simmultaneous performance degradation.

ResNets represent a type of architecture capable of learning **Residual Functions** instead of attempting to perfect full transformations, which allows for implementation of deeper networks while potentially avoiding exploding/vanishing gradients. 

### Learning
 
Instead of attempting to learn the function that transforms the inputs into outputs, a ResNet learns the residual, which represents the difference between te input and output. Essentially, it identifies small transformations for the input, which is usually much easier to learn, as a residual represents an approximation of the actual function.

Choosing the simplest, most shallow version of ResNet allows for a good starting point of expectd performance, as it will attempt to learn approximations of transformations for the DICOM data in the numpy arrays.

### Architecture


<div style="width: 70%; margin: 0 auto; text-align: center;">
    <img src="../z_imgs/ResNet-18-Architecture.png" alt="Hounsfield Units">
</div>


### Implementation

I will start by importing all necessary libraries, as well as the model itself:

In [3]:
# Core library
import torch

# Essentials for development
import torch.nn as nn
import torchvision.models as models

# Data resize (ResNet uses (224,224) inputs)
import torchvision.transforms as transforms

# Allows for paralell batch processing
from torch.utils.data import DataLoader

In [4]:
# Loads the model
resnet18 = models.resnet18(pretrained=True)

d:\Rafa\Anaconda\envs\fibrosis\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Rafa\Anaconda\envs\fibrosis\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The original ResNet18 was designed for ImageNet, which has 1000 possible classes. Since our problem consists of binary classification, the last layer needs the following modification:

In [5]:
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 2)  

In [6]:
# Resizing numpy arrays
transform = transforms.Resize((224, 224))

In order to obtain valid results, the train test split must be done following the recommended data division. Every slice ID starting with "SerieCT" is to be used as a test/validation split.

In [7]:
# Loads the dataframe
# df_fibrosis = pd.read_pickle(r'X:\\RafaelAndre\\pickle_jar_local\\fibrosis_data.pkl')

df_fibrosis = pd.read_pickle(r'D:\Rafa\A1Uni\2semestre\Estágio\fibrosis_data.pkl')

In [8]:
df_fibrosis.head()

,SliceID,SliceData,Class
0,CT-0002-0001.dcm,"[[-2000.0, -2000.0, -2000.0, -2000.0, -2000.0,...",0
1,CT-0002-0002.dcm,"[[-2000.0, -2000.0, -2000.0, -2000.0, -2000.0,...",0
2,CT-0002-0003.dcm,"[[-2000.0, -2000.0, -2000.0, -2000.0, -2000.0,...",0
3,CT-0002-0004.dcm,"[[-2000.0, -2000.0, -2000.0, -2000.0, -2000.0,...",1
4,CT-0002-0005.dcm,"[[-2000.0, -2000.0, -2000.0, -2000.0, -2000.0,...",1


This dataloader expects a Dataset instead of a Dataframe. In order to avoid unnecessary transformations, I will simply be using a simple tensor conversion and the TensorDataset wrapper:

In [ ]:
def adapt_to_resnet(df_original):
    # Generate new dataframe
    df=df_original.copy(deep=True)

    # Transformation pipeline
    transform = transforms.Compose([
        transforms.ToTensor(),  # Converts np array to tensor, automatically adds grayscale dimension (no color in DICOM)
        transforms.Resize((224, 224)),  # ResNet reccomended size 
    ])

    # The apply function executes pseudofunctions with every line as 
    # an argument, which is a more efficient way to iterate changes
    df["SliceData"] = df["SliceData"].apply(lambda x: transform(x))

    return df

In [39]:
df_resnet = adapt_to_resnet(df_fibrosis)

Let's check if the contents changed to tensor and represent an image of size ((224,224)) with 1 grayscale dimension:

In [ ]:
i = random.choice(range(len(df_fibrosis)))

print((df_fibrosis["SliceData"].iloc[i]).shape)
print((df_resnet["SliceData"].iloc[i]).shape)
print("Resolution successfuly compressed to ResNet reccomendations!")

(512, 512)
torch.Size([1, 224, 224])
Resolution successfuly compressed to ResNet reccomendations!


In [50]:
def train_test_fibrosis(df):

    # Selects every line where "SliceID" contains "SerieCT"
    test_df = df[df["SliceID"].str.contains("SerieCT", na=False)]
    train_df = df[~df["SliceID"].str.contains("SerieCT", na=False)]
    return train_df, test_df

In [51]:
train_df, test_df = train_test_fibrosis(df_fibrosis)

In [52]:
criterion = nn.CrossEntropyLoss()  # Loss function 
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.001)

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

num_epochs = 5

for epoch in range(num_epochs):
    resnet18.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


KeyError: 822

In [ ]:
resnet18.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
